In [53]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import namedtuple
QTensor = namedtuple('QTensor', ['tensor', 'scale', 'zero_point'])
def quantize_tensor(x, num_bits=8, min_val=None, max_val=None,imposePow=None,roundType='round'):
    if imposePow is not None:
        scale=2**(-1*imposePow)
    else:
        if not min_val and not max_val: 
            min_val, max_val = x.min(), x.max()
        if min_val!=max_val:
            scale=2**(torch.ceil(torch.log2((max_val-min_val)/(2.**(num_bits)-1.))))
        else:
            scale=2**(torch.ceil(torch.log2((torch.tensor([1e-4]))/(2.**(num_bits)-1.))))
    zero_point=0
    q_x = zero_point + x / scale
    qmin = (2.**(num_bits-1))*(-1.)
    qmax = 2.**(num_bits-1) - 1.
    if roundType=='round':
        q_x.clamp_(qmin, qmax).round_()
    elif roundType=='floor':
        q_x.clamp_(qmin, qmax).floor_()
    
    return QTensor(tensor=q_x, scale=scale, zero_point=zero_point)

def dequantize_tensor(q_x):
    return q_x.scale * (q_x.tensor.float() - q_x.zero_point)

class FakeQuantOp(torch.autograd.Function):
    @staticmethod
    def forward(ctx, w, numBits=4,imposePow=None,roundType='round'):
        x = quantize_tensor(w,num_bits=numBits,imposePow=imposePow,roundType=roundType)
        x = dequantize_tensor(x)
        return x
    @staticmethod
    def backward(ctx, grad_output):
        # straight through estimator
        return grad_output, None, None, None

In [54]:
bn = torch.nn.BatchNorm1d(10)
conv = nn.Conv1d(8, 10, 3, 1)

In [55]:
conv.weight

Parameter containing:
tensor([[[ 0.0182,  0.0218, -0.0378],
         [-0.0876, -0.1781, -0.0784],
         [ 0.0898, -0.1050, -0.1420],
         [-0.0070,  0.0931, -0.1632],
         [-0.1119, -0.0430,  0.0136],
         [ 0.0467,  0.0417, -0.1641],
         [-0.0100,  0.1508,  0.1286],
         [ 0.0775,  0.0924,  0.2001]],

        [[ 0.0254, -0.1487,  0.1017],
         [ 0.1181,  0.1975,  0.1414],
         [ 0.1320, -0.0695, -0.0754],
         [ 0.1980,  0.1950, -0.1130],
         [ 0.0822,  0.1052,  0.0684],
         [-0.0262, -0.0292,  0.1041],
         [ 0.1981,  0.1516, -0.1985],
         [-0.0874,  0.1352, -0.0038]],

        [[-0.0664, -0.0327, -0.1545],
         [-0.1124,  0.0601,  0.1165],
         [ 0.1569,  0.1676, -0.0359],
         [-0.1363,  0.1313,  0.0373],
         [-0.1252, -0.0822, -0.1826],
         [ 0.1955,  0.0219, -0.0948],
         [ 0.1164, -0.2021,  0.0046],
         [ 0.1856, -0.0805, -0.1149]],

        [[ 0.1810,  0.0666,  0.1076],
         [-0.1714,  0.

In [56]:
class ConvBN_Quant(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1,nBits=8):
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.nBits=nBits
        super(ConvBN_Quant, self).__init__()
        self.conv = nn.Conv1d(in_channels,out_channels,kernel_size,stride)
        self.bn = nn.BatchNorm1d(out_channels)
        self.folded = False
        self.quantized = False

    def forward(self,x):
        r=self.bn.weight.unsqueeze(1).unsqueeze(1)
        var=self.bn.running_var.unsqueeze(1).unsqueeze(1)
        
        if not self.folded:
            w=self.conv.weight*r/(torch.sqrt(var+self.bn.eps))
            b=(self.conv.bias-self.bn.running_mean)*self.bn.weight/(torch.sqrt(self.bn.running_var+self.bn.eps))+self.bn.bias
        else:
            w=self.conv.weight
            b=self.conv.bias
        if not self.quantized:        
            w=FakeQuantOp.apply(w,self.nBits)
            b=FakeQuantOp.apply(b,self.nBits)

        x=F.conv1d(x,w,b,self.stride,self.padding)
        return x

    def fold(self):
        r=self.bn.weight.unsqueeze(1).unsqueeze(1)
        var=self.bn.running_var.unsqueeze(1).unsqueeze(1)
        self.conv.weight.data = self.conv.weight.data*r/(torch.sqrt(var+self.bn.eps))
        self.conv.bias.data = (self.conv.bias.data-self.bn.running_mean)*self.bn.weight.data/(torch.sqrt(self.bn.running_var+self.bn.eps))+self.bn.bias.data
        self.folded = True

    def foldAndQuantize(self):
        self.fold()
        self.quantized = True
        self.conv.weight.data = FakeQuantOp.apply(self.conv.weight.data,self.nBits)
        self.conv.bias.data = FakeQuantOp.apply(self.conv.bias.data,self.nBits)

class Linear_Quant(nn.Module):
    def __init__(self,in_features,out_features,nBits=8):
        super(Linear_Quant,self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.nBits = nBits
        self.fc = nn.Linear(in_features,out_features)
        self.quantized = False

    def forward(self,x):
        w = self.fc.weight
        b = self.fc.bias
        if not self.quantized:
            w = FakeQuantOp.apply(w,self.nBits)
            b = FakeQuantOp.apply(b,self.nBits)
        x=F.linear(x,w,b)
        return x

    def quantize(self):
        self.quantized = True
        self.fc.weight.data = FakeQuantOp.apply(self.fc.weight.data,self.nBits)
        self.fc.bias.data = FakeQuantOp.apply(self.fc.bias.data,self.nBits)

In [57]:
class ModelSL_quant(nn.Module):
    def __init__(self,numNeurons=32,nBits=4):
        super(ModelSL_quant, self).__init__()

        self.numNeurons = numNeurons
        self.nBits = nBits

        self.conv1 = ConvBN_Quant(8,10,3,stride=1,padding=0,nBits=nBits)#output 3*13*13
        self.fc1 = Linear_Quant(10 * 3, 80 ,nBits=nBits)
        self.fc2 = Linear_Quant(80, 27,nBits=nBits)
        self.dropout=nn.Dropout(p=0.2)
      
    def forward(self, x):
        x = self.conv1(x)
        x=F.relu6(x)
        x=FakeQuantOp.apply(x,self.nBits,None,'floor')

        x=x.view(-1, 10*3)
        x=self.dropout(x)
        x=self.fc1(x)
        # x=F.relu6(x)
        x=FakeQuantOp.apply(x,self.nBits,None,'floor')
        
        x=self.fc2(x)
        # x=F.relu6(x)
        x=FakeQuantOp.apply(x,self.nBits,None,'floor')
        
        # x=self.fc3(x)
        # x=FakeQuantOp.apply(x,self.nBits,self.nBits-4,'floor')
        
        return x

    def foldAndQuantize(self):
        self.conv1.foldAndQuantize()
        self.fc1.quantize()
        # self.fc2.quantize()
        # self.fc3.quantize()

In [58]:
import os
from torch.utils.data import Dataset, DataLoader
import numpy as np

class NMSLDataset(Dataset):
    def __init__(self, data_path: str):
        data = []
        labels = []

        speakers = os.listdir(data_path)

        for i in speakers:
            labeled_data_files = os.listdir(os.path.join(data_path, i))
            for file in labeled_data_files:
                with open(os.path.join(data_path, i, file), "r") as f:
                    for line in f.readlines()[:-1]:
                        line = line.strip().split(" ")
                        v = [int(i) for i in line[:]]

                        data.append(v)
                        if file[0] in [chr(ord("A") + i) for i in range(26)]:
                            labels.append(torch.tensor(ord(file[0]) - ord("A")))
                        elif "halt" in file:
                            labels.append(torch.tensor(26))
                        elif "random" in file:
                            labels.append(torch.tensor(27))

        data = np.array(data)
        # normalize data
        self.mean = np.int16(np.mean(data, axis=0))
        self.std = np.int16(np.std(data, axis=0))
        data = (data - np.int16(np.mean(data, axis=0))) / np.int16(np.std(data, axis=0))
        # data = (data - np.mean(data, axis=0))

        self.data = []
        self.labels = []

        this_data = []
        this_label = -1
        for i, v in enumerate(data):
            if len(this_data) < 5:
                this_data.append(v)
                this_label = labels[i]
            else:
                self.data.append(torch.tensor(this_data, dtype=torch.float).T)
                self.labels.append(torch.tensor(this_label))

                if labels[i] != this_label:
                    this_data = []
                    this_label = -1
                    continue
                else:
                    this_data.pop(0)
                    this_data.append(v)

    def __getitem__(self, index):
        return self.data[index], self.labels[index]

    def __len__(self):
        return len(self.data)

In [59]:
config = {
    "batch_size": 16,
}

train_set = NMSLDataset("../dataset/train")
train_loader = DataLoader(train_set, batch_size=config["batch_size"], shuffle=True)

# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"

model = ModelSL_quant(nBits=8).to(device)
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.8)

/tmp/ipykernel_4384/2195322645.py:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.labels.append(torch.tensor(this_label))


In [60]:
# from eval_quantize import Classifier

# model_fp32 = Classifier()
# model_fp32.eval()

# model_fp32.qconfig = torch.quantization.get_default_qconfig("fbgemm")
# model_fp32_fused = torch.quantization.fuse_modules(
#     model_fp32, [["conv", "batchnorm"]]
# )
# model_fp32_prepared = torch.quantization.prepare(model_fp32_fused)

# model_fp32_prepared.load_state_dict(torch.load("models/model_quantize.pth"))

# model_int8 = torch.quantization.convert(model_fp32_prepared)

# model.conv1.conv.weight = torch.nn.Parameter(torch.dequantize(model_int8.conv.weight()))
# torch.dequantize(model_int8.conv.weight()) * model_int8.conv.scale

In [61]:
# model.fc1.fc.weight = model_continuous.fc[0].weight

In [62]:
from tqdm import tqdm

for epoch in range(50):
    model.train()

    train_acc = 0
    train_loss = 0

    with tqdm(train_loader, desc=f"Epoch {epoch + 1}") as tepoch:
        for batch in tepoch:
            inputs, labels = batch
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            _, train_pred = torch.max(outputs, 1)
            loss.backward()
            optimizer.step()

            train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
            train_loss = loss.item()

            tepoch.set_postfix(loss=train_loss)
        
        scheduler.step()
        print(f"Train Accuracy: {train_acc / len(train_set)}")

Epoch 1: 100%|██████████| 847/847 [00:02<00:00, 363.63it/s, loss=781]    


Train Accuracy: 0.09587423426083105


Epoch 2: 100%|██████████| 847/847 [00:02<00:00, 360.53it/s, loss=534]    


Train Accuracy: 0.11425197431544763


Epoch 3: 100%|██████████| 847/847 [00:02<00:00, 360.98it/s, loss=1.59e+3]


...

Train Accuracy: 0.10140969813270352


Epoch 13: 100%|██████████| 847/847 [00:02<00:00, 365.92it/s, loss=140]    


Train Accuracy: 0.10318104657170271


Epoch 14:  31%|███▏      | 265/847 [00:00<00:01, 352.67it/s, loss=50.4]


KeyboardInterrupt: 

In [ ]:
torch.save(model, "quant_unnorm.pt")

In [ ]:
model.conv1.conv.weight

Parameter containing:
tensor([[[ 7.4431,  7.6046,  7.4241],
         [ 1.1268,  1.0168,  0.1720],
         [ 2.4845,  3.3358,  4.4562],
         [-2.5527, -2.6465, -2.7437],
         [-7.6747, -7.4491, -7.4075],
         [ 8.4138,  8.3789,  8.3266],
         [-5.4497, -5.6498, -5.5687],
         [ 3.7767,  3.5384,  3.7582]],

        [[ 0.7080,  0.5924,  0.9628],
         [ 0.4951,  0.3349,  0.4743],
         [-0.9440, -0.3481, -0.0550],
         [ 2.1597,  2.0484,  1.7948],
         [ 1.4867,  1.2428,  1.3726],
         [ 2.9807,  2.8316,  2.5310],
         [-0.5800, -1.0828, -0.6880],
         [-8.4748, -8.2104, -8.5670]],

        [[-5.0519, -4.7862, -4.3220],
         [-3.0193, -1.8639, -1.4457],
         [ 2.3664,  2.8255,  2.3234],
         [ 0.7296,  1.1839,  1.4363],
         [ 0.6585,  0.9863,  1.4566],
         [ 0.1180,  0.0963,  0.2170],
         [-4.7629, -4.8583, -4.6756],
         [ 1.8902,  1.8657,  1.5236]],

        [[ 8.6610,  8.3986,  8.9391],
         [-3.2035, -1.

In [ ]:
from eval import Classifier

model_continuous = Classifier()
model_continuous.load_state_dict(torch.load("./models/model.pth"))

<All keys matched successfully>

In [ ]:
model.conv1.conv.weight =  model_continuous.cnn[0].weight

In [ ]:
model_continuous.fc[0].weight

Parameter containing:
tensor([[  9.9009,   6.7030,  12.8561, -17.1859, -15.4959, -14.5686,  10.5488,
           9.6615,  11.1175,  13.5918,  12.2274,  12.0412, -13.4970, -10.8626,
          -7.1948,   8.3417,   8.2107,  12.0162,  -9.4247,  -9.1299,  -9.3204,
          -7.0088,  -6.6695,  -4.7617,  20.4395,  20.8245,  19.5263, -21.5311,
         -22.3276, -24.5059],
        [ -9.8423,  -8.2556,  -8.6066,  18.7477,  18.0189,  18.0065, -13.7677,
         -10.0600, -12.6918, -28.6141, -27.7226, -27.3027,   5.0098,   4.6335,
           1.5405,   4.2506,   6.7063,   5.2010,   8.0726,   5.3810,   2.6762,
          18.4038,  16.9606,  17.1018,  -2.3210,  -3.7809,  -4.4687,   2.4210,
           2.4838,   3.7457],
        [-11.5728,  -8.5998, -15.1555, -15.2280, -13.8639, -15.0411, -10.5546,
          -9.7625, -12.5296,   7.7000,   7.4855,   9.8387, -10.7691,  -9.3761,
         -10.3291, -11.7320,  -6.8195, -12.5887, -11.6014, -10.7547, -13.4409,
          -1.7092,   0.1532,  -3.4158,  14.4693, 

In [ ]:
x = model.conv1(batch[0])
FakeQuantOp.apply(x,4,2,'floor')

tensor([[[ 1.7500,  1.7500,  1.7500],
         [ 1.7500,  1.7500,  1.7500],
         [-1.2500, -1.0000, -0.7500],
         [ 1.7500,  1.7500,  1.7500],
         [-1.7500, -2.0000, -2.0000],
         [-2.0000, -2.0000, -2.0000],
         [ 1.7500,  1.7500,  1.7500],
         [ 1.7500,  1.7500,  1.7500],
         [ 1.7500,  1.7500,  1.7500],
         [ 1.7500,  1.7500,  1.7500]],

        [[-2.0000, -2.0000, -2.0000],
         [ 1.7500,  1.7500,  1.7500],
         [ 0.7500,  0.7500,  0.7500],
         [-2.0000, -2.0000, -2.0000],
         [-2.0000, -2.0000, -2.0000],
         [-2.0000, -2.0000, -2.0000],
         [-1.2500, -1.2500, -0.7500],
         [ 1.7500,  1.7500,  1.7500],
         [ 1.7500,  1.7500,  1.7500],
         [-2.0000, -1.7500, -2.0000]],

        [[ 1.7500,  1.7500,  1.7500],
         [ 1.7500,  1.7500,  1.7500],
         [-0.5000, -0.2500, -0.2500],
         [ 1.7500,  1.7500,  1.7500],
         [-2.0000, -2.0000, -2.0000],
         [-2.0000, -2.0000, -2.0000],
        

In [ ]:
x

tensor([[[  1.8261,   2.2174,   2.5727],
         [  6.2744,   6.3523,   6.4094],
         [ -1.1992,  -0.8839,  -0.5984],
         [  6.9401,   7.0403,   7.0215],
         [ -1.6119,  -2.8514,  -3.6193],
         [-24.9754, -23.9446, -23.6868],
         [  2.2261,   2.3035,   1.9894],
         [  3.1311,   1.9863,   2.0373],
         [ 22.8820,  21.4625,  20.4743],
         [ 13.8359,  13.4738,  13.7487]],

        [[ -6.7216,  -6.6231,  -6.4130],
         [  7.2416,   7.0602,   6.9100],
         [  0.8252,   0.9631,   0.9869],
         [ -8.3983,  -8.9000,  -9.0575],
         [-19.6867, -20.1361, -20.1205],
         [ -2.4598,  -2.5005,  -2.3612],
         [ -1.0218,  -1.1869,  -0.6536],
         [  7.3385,   9.7405,   9.8739],
         [ 36.9013,  36.9882,  37.3386],
         [ -1.8702,  -1.5086,  -2.3235]],

        [[  2.2631,   2.3896,   2.3993],
         [  6.6419,   6.6683,   6.5779],
         [ -0.3333,  -0.1865,  -0.2282],
         [  5.8046,   5.8779,   6.2832],
         [ -

In [ ]:
train_set.mean

array([773, -88, 257, 318, 324, 334, 340, 307], dtype=int16)

In [ ]:
train_set.std

array([497, 253, 272,  19,  45,  43,  36,  41], dtype=int16)

In [ ]:
max_data = max([torch.max(i) for i in train_set.data])
min_data = min([torch.min(i) for i in train_set.data])

In [ ]:
max_data, min_data

(tensor(5.0435), tensor(-3.8972))

In [ ]:
torch.max(model.conv1(batch[0])), torch.std(model.conv1(batch[0])), torch.mean(model.conv1(batch[0]))

(tensor(8.7772, grad_fn=<MaxBackward1>),
 tensor(1.9489, grad_fn=<StdBackward0>),
 tensor(0.3777, grad_fn=<MeanBackward0>))

tensor(6., grad_fn=<MaxBackward1>)

In [64]:
torch.max(model.conv1.conv.weight)

tensor(7.0318, grad_fn=<MaxBackward1>)

In [65]:
model.conv1.conv.weight.shape

torch.Size([10, 8, 3])

In [67]:
model.conv1.conv.bias

Parameter containing:
tensor([-30.3771, -28.3105, -30.9589, -30.7700, -19.0844, -27.0349, -27.8618,
        -22.6335, -29.6849, -28.1582], requires_grad=True)